# LiDAR–Camera–IMU Odometry Pipeline (Practical)
This notebook bundles the scripts you were using (`data.py`, `extract.py`, `structure.py`, `training.py`, `model.py`, `inference_lidar_local.py`) into one practical, runnable workflow.
- **Edit paths** (`base_dir`, `bag_file`, `url`) for your machine (Windows/G: drive or Colab).
- Cells are independent — run only what you need.
- GPU is **recommended** for training/inference.

In [4]:

import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

# ✅ EDIT THIS for your machine
BASE_DIR = r"G:\backup\papers\Dataset\hidrive_file"  # Windows path
os.makedirs(BASE_DIR, exist_ok=True)
training_csv_path = r"G:\backup\papers\Dataset\hidrive_file\training_data.csv"
print("Base dir:", BASE_DIR)


Base dir: G:\backup\papers\Dataset\hidrive_file


## 1. Download HiDrive file (optional)
Uses `aria2c` like your `data.py`. Skip if you've already downloaded the `.bag`.

In [5]:

import subprocess

# ⚠️ EDIT THESE:
url = "https://my.hidrive.com/api/file?attachment=true&pid=b1584010878.594&access_token=REPLACE_ME"
output_file = os.path.join(BASE_DIR, "hidrive_file.bag")

cmd = [
    "aria2c",
    url,
    "-o", output_file,
    "-x", "16",
    "-s", "16",
    "-k", "1M",
    "--console-log-level=error",
    "--summary-interval=5",
]

print("Downloading to:", output_file)
try:
    subprocess.run(cmd, check=True)
    print("✅ Download completed")
except Exception as e:
    print("❌ Download failed:", e)


❌ Download failed: Command '['aria2c', 'https://my.hidrive.com/api/file?attachment=true&pid=b1584010878.594&access_token=REPLACE_ME', '-o', 'G:\\backup\\papers\\Dataset\\hidrive_file\\hidrive_file.bag', '-x', '16', '-s', '16', '-k', '1M', '--console-log-level=error', '--summary-interval=5']' returned non-zero exit status 24.


## 2. Extract ROS bag to folders
This is your `extract.py` adapted for notebook. Requires ROS Python packages (`rosbag`, `sensor_msgs`, `cv_bridge`). If you're on Windows without ROS, you can skip this and place extracted data manually.

In [ ]:

import csv
import yaml
from pathlib import Path

# If ROS not available, this cell will raise — that's expected.
try:
    import rosbag
    from sensor_msgs import point_cloud2
    from cv_bridge import CvBridge
    import cv2
    import open3d as o3d
except ImportError as e:
    print("ROS/Open3D deps missing:", e)

bag_file = os.path.join(BASE_DIR, "hidrive_file.bag")

out_dirs = {
    "left": os.path.join(BASE_DIR, "images", "left"),
    "right": os.path.join(BASE_DIR, "images", "right"),
    "lidar": os.path.join(BASE_DIR, "lidar"),
    "camera_info": os.path.join(BASE_DIR, "camera_info"),
    "imu": os.path.join(BASE_DIR, "imu"),
    "tf": os.path.join(BASE_DIR, "tf"),
}
for d in out_dirs.values():
    os.makedirs(d, exist_ok=True)

print("[INFO] Opening bag:", bag_file)

try:
    bag = rosbag.Bag(bag_file)
except Exception as e:
    print("❌ Couldn't open bag:", e)
    bag = None

imu_topics = ['/imu/data', '/imu/dq', '/imu/dv', '/imu/mag', '/imu/time_ref']
imu_files = {}
imu_writers = {}
for t in imu_topics:
    fname = os.path.join(out_dirs['imu'], t.strip('/').replace('/', '_') + '.csv')
    f = open(fname, 'w', newline='')
    w = csv.writer(f)
    w.writerow(['timestamp', 'data...'])
    imu_files[t] = f
    imu_writers[t] = w

left_images = []
right_images = []
lidar_scans = []

if bag is not None:
    bridge = CvBridge()
    print("[INFO] Starting extraction...")
    for topic, msg, t in tqdm(bag.read_messages(), total=bag.get_message_count()):
        ts = t.to_nsec()

        if topic == '/bf_lidar/points_raw':
            pcd_name = f"{ts}.pcd"
            pcd_path = os.path.join(out_dirs["lidar"], pcd_name)
            try:
                pts = list(point_cloud2.read_points(msg, skip_nans=True))
                pc = o3d.geometry.PointCloud()
                pc.points = o3d.utility.Vector3dVector([[p[0], p[1], p[2]] for p in pts])
                o3d.io.write_point_cloud(pcd_path, pc)
                lidar_scans.append((ts, pcd_name))
            except Exception as e:
                print("LiDAR conv failed:", e)

        elif topic in imu_topics:
            writer = imu_writers[topic]
            row = [ts]
            if hasattr(msg, '__slots__'):
                for field in msg.__slots__:
                    val = getattr(msg, field)
                    if hasattr(val, '__slots__'):
                        for sub in val.__slots__:
                            row.append(getattr(val, sub))
                    else:
                        row.append(val)
            writer.writerow(row)

        elif topic == '/stereo/left/image_rect':
            img = bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')
            name = f"{ts}.png"
            path = os.path.join(out_dirs["left"], name)
            cv2.imwrite(path, img)
            left_images.append((ts, name))

        elif topic == '/stereo/right/image_rect':
            img = bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')
            name = f"{ts}.png"
            path = os.path.join(out_dirs["right"], name)
            cv2.imwrite(path, img)
            right_images.append((ts, name))

    # associations
    assoc_path = os.path.join(BASE_DIR, "associations.txt")
    with open(assoc_path, "w") as f:
        for l, r, p in zip(left_images, right_images, lidar_scans):
            f.write(f"{l[0]} {l[1]} {r[0]} {r[1]} {p[0]} {p[1]}\n")

    for f in imu_files.values():
        f.close()

    bag.close()
    print("✅ Extraction complete")
else:
    print("⚠️ Skipped ROS extraction (no bag)")


## 3. View dataset folder structure

In [3]:

import os
from collections import defaultdict

def print_folder_tree_limited(startpath, prefix=""):
    try:
        items = os.listdir(startpath)
    except PermissionError:
        print(prefix + "└── [Permission Denied]")
        return

    file_groups = defaultdict(list)
    dirs = []

    for item in items:
        path = os.path.join(startpath, item)
        if os.path.isdir(path):
            dirs.append(item)
        else:
            ext = os.path.splitext(item)[1] or "no_ext"
            file_groups[ext].append(item)

    for i, d in enumerate(sorted(dirs)):
        path = os.path.join(startpath, d)
        connector = "└── " if i == len(dirs) - 1 and not file_groups else "├── "
        print(prefix + connector + d)
        extension = "    " if i == len(dirs) - 1 and not file_groups else "│   "
        print_folder_tree_limited(path, prefix + extension)

    for ext, files in file_groups.items():
        for idx, file in enumerate(sorted(files)):
            if idx >= 2:
                if idx == 2:
                    print(prefix + f"└── +{len(files)-2} more {ext} files")
                break
            connector = "└── " if idx == len(files)-1 or len(files) <= 2 else "├── "
            print(prefix + connector + file)

print("📁 Folder structure for:", BASE_DIR)
print_folder_tree_limited(BASE_DIR)



📁 Folder structure for: G:\backup\papers\Dataset\hidrive_file
├── camera_info
│   ├── 1625657732290284015.yaml
│   ├── 1625657732323859820.yaml
│   └── +16943 more .yaml files
├── images
│   ├── left
│   │   ├── 1625657732290284015.png
│   │   ├── 1625657732290284032.png
│   │   └── +16943 more .png files
│   └── right
│       ├── 1625657732290284015.png
│       ├── 1625657732323859820.png
│       └── +16879 more .png files
├── imu
│   ├── imu_data.csv
│   ├── imu_dq.csv
│   └── +3 more .csv files
├── imu_data_for_training
│   ├── 1.6256577323119644e+18_to_1.6256577325274007e+18.npy
│   ├── 1.6256577325274007e+18_to_1.6256577327408215e+18.npy
│   └── +107 more .npy files
├── lidar
│   ├── 1625657732311964358.pcd
│   ├── 1625657732527400610.pcd
│   └── +2654 more .pcd files
├── tf
└── associations.txt
└── bf_lidar-points_raw.csv
└── training_data.csv
├── lidar_trajectory_2d.png
├── lidar_trajectory_3d.png
└── +2 more .png files
└── liv_odometry_model.pth


## 4. Create training CSV using LiDAR ICP
This is your `training.py` compacted. It reads `associations.txt` and writes `training_data.csv`.

In [20]:

import open3d as o3d
from scipy.spatial.transform import Rotation

lidar_dir = os.path.join(BASE_DIR, "lidar")
associations_path = os.path.join(BASE_DIR, "associations.txt")
training_csv_path = os.path.join(BASE_DIR, "training_data.csv")

def load_associations(path):
    pairs = []
    if not os.path.exists(path):
        print("❌ associations.txt not found:", path)
        return pairs
    with open(path, "r") as f:
        for line in f:
            if line.startswith("#") or not line.strip():
                continue
            parts = line.split()
            if len(parts) == 6:
                img_l, img_r, pcd = os.path.basename(parts[1]), os.path.basename(parts[3]), os.path.basename(parts[5])
                pairs.append((img_l, img_r, pcd))
    print(f"Loaded {len(pairs)} associations")
    return pairs

def matrix_to_6dof(T):
    t = T[:3, 3]
    r = Rotation.from_matrix(T[:3, :3]).as_euler('xyz', degrees=False)
    return np.concatenate((t, r))

def estimate_lidar_odometry(pcd_src, pcd_tgt):
    src = o3d.io.read_point_cloud(pcd_src)
    tgt = o3d.io.read_point_cloud(pcd_tgt)
    if not src.has_points() or not tgt.has_points():
        return None

    voxel = 0.1
    src_d = src.voxel_down_sample(voxel)
    tgt_d = tgt.voxel_down_sample(voxel)
    src_d.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
    tgt_d.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))

    reg = o3d.pipelines.registration.registration_icp(
        src_d, tgt_d, 0.5, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
    )
    if reg.fitness < 0.1:
        return None
    return reg.transformation

pairs = load_associations(associations_path)
rows = []

if len(pairs) > 1:
    print("Generating training_data.csv ...")
    for i in tqdm(range(len(pairs)-1)):
        _, _, pcd_i = pairs[i]
        _, _, pcd_j = pairs[i+1]

        pcd_path_i = os.path.join(lidar_dir, pcd_i)
        pcd_path_j = os.path.join(lidar_dir, pcd_j)

        if not (os.path.exists(pcd_path_i) and os.path.exists(pcd_path_j)):
            continue

        T = estimate_lidar_odometry(pcd_path_i, pcd_path_j)
        if T is None:
            continue
        pose = matrix_to_6dof(T)
        rows.append([pcd_i, pcd_j, *pose.tolist()])

    if rows:
        df = pd.DataFrame(rows, columns=["source_pcd","target_pcd","tx","ty","tz","rx","ry","rz"])
        df.to_csv(training_csv_path, index=False)
        print("✅ Saved:", training_csv_path, "->", len(df), "samples")
    else:
        print("⚠️ No rows generated (ICP failed).")
else:
    print("⚠️ Not enough associations to build dataset.")


Loaded 2656 associations
Generating training_data.csv ...


 63%|██████▎   | 1678/2655 [1:16:02<44:16,  2.72s/it]  


KeyboardInterrupt: 

## 5. Multimodal Odometry Model (LiDAR + IMU + Vision)
This is your model (from `model.py`) in simplified form, with training loop.

In [18]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.amp import autocast, GradScaler
import cv2

IMG_H, IMG_W = 128, 128
N_POINTS = 2048
IMU_SAVE_DIR = os.path.join(BASE_DIR, "imu_data_for_training")
os.makedirs(IMU_SAVE_DIR, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

def load_pcd(path, num_points=N_POINTS):
    try:
        pcd = o3d.io.read_point_cloud(path)
        pts = np.asarray(pcd.points, dtype=np.float32)
        if pts.shape[0] > num_points:
            idx = np.random.choice(pts.shape[0], num_points, replace=False)
            pts = pts[idx]
        elif pts.shape[0] < num_points and pts.shape[0] > 0:
            idx = np.random.choice(pts.shape[0], num_points - pts.shape[0], replace=True)
            pts = np.vstack((pts, pts[idx]))
        elif pts.shape[0] == 0:
            pts = np.zeros((num_points, 3), dtype=np.float32)
        return pts
    except:
        return np.zeros((num_points, 3), dtype=np.float32)

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        return np.zeros((IMG_H, IMG_W, 3), dtype=np.float32)
    img = cv2.resize(img, (IMG_W, IMG_H)).astype(np.float32) / 255.0
    return img

class SimpleLIDARDataset(Dataset):
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.lidar_dir = os.path.join(BASE_DIR, "lidar")
        self.image_dir = os.path.join(BASE_DIR, "images", "left")
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        src_pc = load_pcd(os.path.join(self.lidar_dir, row['source_pcd']))
        tgt_pc = load_pcd(os.path.join(self.lidar_dir, row['target_pcd']))
        pc = np.vstack((src_pc, tgt_pc)).astype(np.float32)
        pc -= pc.mean(axis=0, keepdims=True)
        pc = torch.tensor(pc, dtype=torch.float32).permute(1, 0)

        # dummy imu
        imu = torch.zeros((1, 6), dtype=torch.float32)

        # dummy image pair
        src_img = load_image(os.path.join(self.image_dir, row.get('source_img', '')))
        tgt_img = load_image(os.path.join(self.image_dir, row.get('target_img', '')))
        img = np.concatenate((src_img, tgt_img), axis=2)
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)

        pose = torch.tensor(row[['tx','ty','tz','rx','ry','rz']].values.astype(np.float32))
        return pc, imu, img, pose

def collate_fn(batch):
    pcs = [b[0] for b in batch]
    imus = [b[1] for b in batch]
    imgs = [b[2] for b in batch]
    poses = [b[3] for b in batch]
    pcs = torch.stack(pcs)
    imgs = torch.stack(imgs)
    poses = torch.stack(poses)
    imus = pad_sequence(imus, batch_first=True, padding_value=0.0)
    return pcs, imus, imgs, poses

class LIV_OdometryNet(nn.Module):
    def __init__(self, d_model=512, n_head=8):
        super().__init__()
        self.lidar_fe = nn.Sequential(
            nn.Conv1d(3, 64, 1), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Conv1d(64, 128, 1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 1024, 1), nn.BatchNorm1d(1024), nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        self.imu_rnn = nn.GRU(6, 128, 2, batch_first=True, dropout=0.1)
        self.visual_fe = nn.Sequential(
            nn.Conv2d(6, 32, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(128, 256, 3, 2, 1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.lidar_proj = nn.Linear(1024, d_model)
        self.imu_proj = nn.Linear(128, d_model)
        self.visual_proj = nn.Linear(256, d_model)

        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, batch_first=True)
        self.fusion = nn.TransformerEncoder(enc_layer, num_layers=2)
        self.head = nn.Sequential(
            nn.Linear(d_model*3, 1024), nn.ReLU(),
            nn.Linear(1024, 256), nn.ReLU(),
            nn.Linear(256, 6)
        )

    def forward(self, pc, imu, img):
        B = pc.size(0)
        lidar_feat = self.lidar_fe(pc).view(B, -1)
        _, h_n = self.imu_rnn(imu)
        imu_feat = h_n[-1]
        vis_feat = self.visual_fe(img).view(B, -1)

        tok_l = self.lidar_proj(lidar_feat)
        tok_i = self.imu_proj(imu_feat)
        tok_v = self.visual_proj(vis_feat)

        tokens = torch.stack([tok_l, tok_i, tok_v], dim=1)
        fused = self.fusion(tokens).reshape(B, -1)
        out = self.head(fused)
        return out

training_csv = training_csv_path
if os.path.exists(training_csv):
    ds = SimpleLIDARDataset(training_csv)
    loader = DataLoader(ds, batch_size=4, shuffle=True, collate_fn=collate_fn)
    model = LIV_OdometryNet().to(device)
    criterion = nn.SmoothL1Loss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scaler = GradScaler()

    print("Starting 1 short epoch (debug)...")
    model.train()
    for pc, imu, img, pose in tqdm(loader):
        pc, imu, img, pose = pc.to(device), imu.to(device), img.to(device), pose.to(device)
        optimizer.zero_grad()
        with autocast(device_type=device.type, dtype=torch.float16 if device.type=='cuda' else torch.bfloat16):
            preds = model(pc, imu, img)
            loss = criterion(preds, pose)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        print("loss:", loss.item())
    torch.save(model.state_dict(), os.path.join(BASE_DIR, "liv_odometry_model.pth"))
    print("✅ Saved model to", os.path.join(BASE_DIR, "liv_odometry_model.pth"))
else:
    print("⚠️ training_data.csv not found, skip training")


Using device: cuda
Starting 1 short epoch (debug)...


  8%|▊         | 2/25 [00:01<00:10,  2.10it/s]

loss: 0.0027224537916481495
loss: 0.03581151366233826


 16%|█▌        | 4/25 [00:01<00:05,  3.85it/s]

loss: 0.006726098246872425
loss: 0.006310615222901106


...

loss: 0.003668104065582156
loss: 0.0033381031826138496


 96%|█████████▌| 24/25 [00:04<00:00,  7.33it/s]

loss: 0.01815703511238098
loss: 0.0024997908622026443


100%|██████████| 25/25 [00:04<00:00,  5.61it/s]

loss: 0.04777729511260986
✅ Saved model to G:\backup\papers\Dataset\hidrive_file\liv_odometry_model.pth


## 6. Inference + Trajectory Plot
Loads the saved model and the CSV, runs inference in sequence, and plots predicted vs. chained GT.

In [ ]:

import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

MODEL_PATH = os.path.join(BASE_DIR, "liv_odometry_model.pth")

def dof6_to_matrix(pose_6d):
    T = np.eye(4, dtype=np.float32)
    t = pose_6d[0:3]
    r_euler = pose_6d[3:6]
    r = Rotation.from_euler('xyz', r_euler, degrees=False)
    T[:3, :3] = r.as_matrix().astype(np.float32)
    T[:3, 3] = t
    return T

if os.path.exists(MODEL_PATH) and os.path.exists(training_csv_path):
    ds_inf = SimpleLIDARDataset(training_csv_path)
    inf_loader = DataLoader(ds_inf, batch_size=4, shuffle=False, collate_fn=collate_fn)

    model = LIV_OdometryNet().to(device)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.eval()

    pred_traj = [np.zeros(3, dtype=np.float32)]
    gt_traj = [np.zeros(3, dtype=np.float32)]
    cur_pred = np.eye(4, dtype=np.float32)
    cur_gt = np.eye(4, dtype=np.float32)

    with torch.no_grad():
        for pc, imu, img, pose_6d in tqdm(inf_loader, desc="[Inference]"):
            pc, imu, img = pc.to(device), imu.to(device), img.to(device)
            preds = model(pc, imu, img).cpu().numpy()
            gts = pose_6d.numpy()
            for p6, g6 in zip(preds, gts):
                Tp = dof6_to_matrix(p6)
                Tg = dof6_to_matrix(g6)
                cur_pred = cur_pred @ Tp
                cur_gt = cur_gt @ Tg
                pred_traj.append(cur_pred[:3, 3].copy())
                gt_traj.append(cur_gt[:3, 3].copy())

    pred_traj = np.array(pred_traj)
    gt_traj = np.array(gt_traj)

    plt.figure(figsize=(8,8))
    plt.plot(gt_traj[:,0], gt_traj[:,1], label="Ground Truth")
    plt.plot(pred_traj[:,0], pred_traj[:,1], "--", label="Predicted")
    plt.legend()
    plt.title("Trajectory (XY)")
    plt.axis("equal")
    plt.grid(True)
    plt.show()
else:
    print("⚠️ Model or CSV missing — run training first.")


Using device: cuda


C:\Users\gsing\AppData\Local\Temp\ipykernel_8600\2087450061.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=d

⚠️ Could not load model weights: Error(s) in loading state_dict for LIV_OdometryNet:
	Missing key(s) in state_dict: "img_cnn.0.weight", "img_cnn.0.bias", "img_cnn.2.weight", "img_cnn.2.bias", "pc_fc.0.weight", "pc_fc.0.bias", "pc_fc.2.weight", "pc_fc.2.bias", "imu_fc.0.weight", "imu_fc.0.bias", "imu_fc.2.weight", "imu_fc.2.bias", "fusion.0.weight", "fusion.0.bias", "fusion.2.weight", "fusion.2.bias". 
	Unexpected key(s) in state_dict: "vision.feat.0.weight", "vision.feat.0.bias", "vision.feat.2.weight", "vision.feat.2.bias", "vision.feat.4.weight", "vision.feat.4.bias", "vision.feat.6.weight", "vision.feat.6.bias", "lidar.mlp.0.weight", "lidar.mlp.0.bias", "lidar.mlp.2.weight", "lidar.mlp.2.bias", "lidar.mlp.4.weight", "lidar.mlp.4.bias", "cross1.attn_v2l.in_proj_weight", "cross1.attn_v2l.in_proj_bias", "cross1.attn_v2l.out_proj.weight", "cross1.attn_v2l.out_proj.bias", "cross1.attn_l2v.in_proj_weight", "cross1.attn_l2v.in_proj_bias", "cross1.attn_l2v.out_proj.weight", "cross1.attn_l2v

[Inference]:   0%|          | 0/25 [00:00<?, ?it/s]


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [11]:
import open3d as o3d
import numpy as np

# Assume gt_xy and pred_xy are numpy arrays of shape (N, 2)
# If you already have them, just skip this dummy example.
# Otherwise, load from your CSV or the arrays you computed.
# gt_xy = np.load("gt.npy")
# pred_xy = np.load("pred.npy")

# Example: random dummy data (REMOVE this and use your real gt_xy, pred_xy)
# gt_xy = np.cumsum(np.random.randn(200, 2), axis=0)
# pred_xy = gt_xy + np.random.randn(200, 2) * 0.5

# Convert 2D XY trajectories to 3D (Z=0)
gt_xyz = np.column_stack((gt_xy, np.zeros(len(gt_xy))))
pred_xyz = np.column_stack((pred_xy, np.zeros(len(pred_xy))))

# Create line sets for both
def make_line_set(points, color):
    lines = [[i, i+1] for i in range(len(points)-1)]
    colors = [color for _ in lines]
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

# Ground truth: green, Predicted: red
gt_lines = make_line_set(gt_xyz, [0, 1, 0])
pred_lines = make_line_set(pred_xyz, [1, 0, 0])

# Coordinate frame (for reference)
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)

# Visualize
o3d.visualization.draw_geometries([gt_lines, pred_lines, origin],
                                  window_name="Odometry Trajectory (Open3D)",
                                  width=900, height=700)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


NameError: name 'gt_xy' is not defined